In [1]:
import os
from pathlib import Path
from collections import defaultdict

# Ruta al dataset
DATA_PATH = './Datasets/Cards/train'

# Contador de ejemplos por clase
class_counts = defaultdict(int)
total_images = 0

# Recorrer todas las carpetas (clases)
for class_folder in sorted(Path(DATA_PATH).iterdir()):
    if class_folder.is_dir():
        # Contar imágenes (jpg y png)
        num_images = len(list(class_folder.glob('*.jpg'))) + len(list(class_folder.glob('*.png')))
        class_counts[class_folder.name] = num_images
        total_images += num_images

# Mostrar resultados
print(f"{'Clase':<30} {'Cantidad':>10}")
print("=" * 42)

for class_name, count in sorted(class_counts.items()):
    print(f"{class_name:<30} {count:>10}")

print("=" * 42)
print(f"{'TOTAL':<30} {total_images:>10}")
print(f"\nNúmero de clases: {len(class_counts)}")
print(f"Promedio de imágenes por clase: {total_images / len(class_counts):.1f}")

Clase                            Cantidad
ace of clubs                          120
ace of diamonds                       129
ace of hearts                         171
ace of spades                         181
eight of clubs                        138
eight of diamonds                     159
eight of hearts                       152
eight of spades                       135
five of clubs                         154
five of diamonds                      151
five of hearts                        136
five of spades                        158
four of clubs                         157
four of diamonds                      114
four of hearts                        154
four of spades                        140
jack of clubs                         171
jack of diamonds                      160
jack of hearts                        168
jack of spades                        182
joker                                 126
king of clubs                         128
king of diamonds                  

In [2]:
# Ordenar por cantidad de imágenes de mayor a menor
sorted_classes = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)

print(f"{'Clase':<30} {'Cantidad':>10}")
print("=" * 42)
for name, cnt in sorted_classes:
    print(f"{name:<30} {cnt:>10}")
print("=" * 42)
print(f"{'TOTAL':<30} {total_images:>10}")

Clase                            Cantidad
jack of spades                        182
ace of spades                         181
nine of spades                        176
ace of hearts                         171
jack of clubs                         171
six of clubs                          171
jack of hearts                        168
seven of spades                       165
queen of diamonds                     163
jack of diamonds                      160
eight of diamonds                     159
five of spades                        158
queen of spades                       158
six of spades                         158
ten of spades                         158
three of diamonds                     158
four of clubs                         157
two of hearts                         155
two of spades                         155
five of clubs                         154
four of hearts                        154
queen of clubs                        154
eight of hearts                   

In [2]:
!ls

'03a - SingleDetection-Tools'   SingleDetection-Tools   checkpoints_vgg16
 CNN-Cards		        archive		        logs
 DL-HW-1		        checkpoints_mobilenet   logs_scratch
 Lab04-Segmentation	        checkpoints_resnet50    prueba.ipynb
 Labs			        checkpoints_scratch     requirements.txt


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Verificar que el modelo existe
model_path = 'CNN-Cards/Models/VGG16_finetuned.h5'
if not os.path.exists(model_path):
    print(f'❌ Error: El modelo no existe en {model_path}')
    print(f'\nModelos disponibles en Models/:')
    for f in os.listdir('Models'):
        if f.endswith('.h5'):
            print(f'  - {f}')
else:
    # Configurar generador de test
    test_generator = ImageDataGenerator(rescale=1.0/255)
    
    test_dataset = test_generator.flow_from_directory(
        'CNN-Cards/Datasets/Cards/test',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    
    # Cargar modelo VGG16 con safe_mode para versiones incompatibles
    print('Cargando modelo VGG16...')
    try:
        # Intentar cargar con safe_mode (TF 2.16+)
        try:
            vgg_model = tf.keras.models.load_model(model_path, compile=False, safe_mode=False)
        except TypeError:
            # Si safe_mode no está disponible, usar método antiguo
            vgg_model = tf.keras.models.load_model(model_path, compile=False)
        
        # Re-compilar manualmente
        vgg_model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        print('✓ Modelo cargado exitosamente')
        print(f'Arquitectura: {vgg_model.name if hasattr(vgg_model, "name") else "VGG16"}')
        
        # Evaluar
        print('\nEvaluando en test set...')
        loss, accuracy = vgg_model.evaluate(test_dataset, verbose=1)
        
        print(f'\n{"="*50}')
        print(f'VGG16 Fine-tuned Results:')
        print(f'{"="*50}')
        print(f'Test Loss:     {loss:.4f}')
        print(f'Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)')
        print(f'{"="*50}')
        
    except Exception as e:
        print(f'❌ Error al cargar o evaluar el modelo:')
        print(f'   {type(e).__name__}: {str(e)}')
        print(f'\n💡 Solución alternativa: Intentando reconstruir el modelo...')
        
        # Alternativa: cargar solo los pesos
        try:
            from tensorflow.keras import regularizers
            
            # Reconstruir arquitectura VGG16
            base_vgg16 = tf.keras.applications.VGG16(
                include_top=False,
                weights=None,  # No cargar pesos de ImageNet
                input_shape=(224, 224, 3)
            )
            
            inputs = tf.keras.layers.Input(shape=(224, 224, 3))
            x = base_vgg16(inputs)
            x = tf.keras.layers.GlobalAveragePooling2D()(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
            x = tf.keras.layers.Dropout(0.5)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
            x = tf.keras.layers.Dropout(0.4)(x)
            x = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
            x = tf.keras.layers.Dropout(0.3)(x)
            outputs = tf.keras.layers.Dense(53, activation='softmax')(x)
            
            vgg_model = tf.keras.Model(inputs=inputs, outputs=outputs)
            
            # Cargar solo los pesos
            vgg_model.load_weights(model_path)
            
            vgg_model.compile(
                optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )
            
            print('✓ Modelo reconstruido y pesos cargados exitosamente')
            
            # Evaluar
            print('\nEvaluando en test set...')
            loss, accuracy = vgg_model.evaluate(test_dataset, verbose=1)
            
            print(f'\n{"="*50}')
            print(f'VGG16 Fine-tuned Results (reconstruido):')
            print(f'{"="*50}')
            print(f'Test Loss:     {loss:.4f}')
            print(f'Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)')
            print(f'{"="*50}')
            
        except Exception as e2:
            print(f'❌ Tampoco funcionó la reconstrucción:')
            print(f'   {type(e2).__name__}: {str(e2)}')
            print(f'\n⚠️  Es posible que necesites re-entrenar el modelo con la versión actual de TensorFlow.')

Found 265 images belonging to 53 classes.
Cargando modelo VGG16...
❌ Error al cargar o evaluar el modelo:
   TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer_1'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

💡 Solución alternativa: Intentando reconstruir el modelo...
Cargando modelo VGG16...
❌ Error al cargar o evaluar el modelo:
   TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_layer_1'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

💡 Solución alternativa: Intentando reconstruir el modelo...


2025-12-09 21:12:37.236655: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-09 21:12:37.336970: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-09 21:12:37.337008: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-09 21:12:37.339485: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-09 21:12:37.339535: I external/local_xla/xla/stream_executor

❌ Tampoco funcionó la reconstrucción:
   ValueError: Cannot assign value to variable ' block1_conv1/kernel:0': Shape mismatch.The variable shape (3, 3, 3, 64), and the assigned value shape (512, 256, 3, 3) are incompatible.

⚠️  Es posible que necesites re-entrenar el modelo con la versión actual de TensorFlow.
